## DOcplex modelling 案例

In [2]:
from docplex.mp.model import Model

### 一个NLP(non-linear programming)例子
$$
\text{Maximize } Z = x_1 + x_2
$$
$$
\text{Subject to:}
$$
$$
\begin{align}
1. & \quad x_1 + x_2 \leq 10 \\
& \quad x_1 \cdot x_2 \geq 10 \\
3. & \quad x_1 \geq 0 \\
& \quad x_2 \geq 0 \\
\end{align}
$$

In [3]:

# 创建一个模型
mdl = Model(name="this is a demo ") # 创建一个模型
# 定义决策变量
x1 = mdl.continuous_var(name = 'x1',lb=0) # 先定义决策变量,在定义目标函数
x2 = mdl.continuous_var(name = "x2",lb=0)
# 定义目标函数
mdl.maximize(x1 + x2)
# 定义约束
mdl.add_constraint(x1 + x2 <= 10)
mdl.add_constraint(x1*x2 >= 10)
# 求解模型
solution = mdl.solve()
# 输出结果
if solution:
    print("solution is found")
    print(f"x1:{x1.solution_value},x2:{x2.solution_value}")
    print(f"objective value:{mdl.objective_value}")
else:
    print("solution is not found")


solution is found
x1:4.9999999966802555,x2:4.9999999966802555
objective value:9.999999993360511


### 一个IP例子

In [4]:
mdl = Model(name="Integer Programming")
x1 = mdl.integer_var(name="x1")
x2 = mdl.integer_var(name="x2")
mdl.maximize(x1 + x2)
mdl.add_constraint(x1 + x2 <= 10)
mdl.add_constraint(x1*x2 >= 10)
solution = mdl.solve()
print(solution)


solution for: Integer Programming
objective: 10
x1=8
x2=2



### MILP

In [5]:
from docplex.mp.model import Model

model = Model(name="mixed_integer_programming_example")

# 定义决策变量（一个连续，一个整数）
x1 = model.continuous_var(name="x1")
x2 = model.integer_var(name="x2")

# 设置目标函数
model.maximize(5 * x1 + 3 * x2)

# 添加约束条件
model.add_constraint(2 * x1 + x2 <= 100)
model.add_constraint(x1 + 2 * x2 <= 80)

# 求解
solution = model.solve()
if solution:
    print(f"x1 = {x1.solution_value}, x2 = {x2.solution_value}")


x1 = 40.0, x2 = 20.0


### QP

In [6]:
from docplex.mp.model import Model

model = Model(name="quadratic_programming_example")

# 定义决策变量
x1 = model.continuous_var(name="x1")
x2 = model.continuous_var(name="x2")

# 设置二次目标函数
model.minimize(x1 * x1 + x2 * x2 + 2 * x1 * x2)

# 添加约束条件
model.add_constraint(x1 + x2 >= 5)

# 求解
solution = model.solve()
if solution:
    print(f"x1 = {x1.solution_value}, x2 = {x2.solution_value}")


x1 = 0, x2 = 5.0


## Advanced Method

### 加权和法

原理:
将多个目标函数通过加权系统合并为一个单一目标函数，权重反映每个目标的相对重要性。

数学形式:
$$ \text{Maximize } w_1f_1(x) + w_2f_2(x) $$

其中:
- $w_1, w_2$: 权重，$w_1 + w_2 = 1$ (通常归一化)
- 假设 $f_1$ 是利润，$f_2$ 是成本，则可能需要取负值（如 $-f_2$）统一方向。

示例:
问题：最大化利润 $5x_1 + 3x_2$，同时最小化成本 $2x_1 + 4x_2$

约束：$x_1 + x_2 \leq 10$

In [7]:
from docplex.mp.model import Model
mdl = Model(name="加权和法")

x1 = mdl.continuous_var(name='x1',lb=0)
x2 = mdl.continuous_var(name="x2",lb=0)

objective1 = 5*x1+3*x2
objective2 = 2*x1+4*x2

w1,w2 = 0.7,0.3 # 分配权重

combined_objective = w1*objective1 - w2*objective2 # 利润-成本

mdl.maximize(combined_objective)


mdl.add_constraint(x1+x2<=10)

solution = mdl.solve()
if solution:
    print(f"x1 = {x1.solution_value}, x2 = {x2.solution_value}")
    print(f"Profit = {x1.solution_value}, Cost = {x2.solution_value}")
    print(f"Combined Objective = {mdl.objective_value}")



x1 = 10.0, x2 = 0
Profit = 10.0, Cost = 0
Combined Objective = 29.0


### 分层优化法 *Lexicographic Method*

当目标有明确的优先级时，可以使用分层优化法，先优化最高优先级的目标，然后在保持其最优解的前提下优化次级目标。
* 先优化最高优先级目标，得到最优值。
* 将该目标值作为约束，优化次级目标。
ex: 先最大化利润$5x_1+3x_2$,然后在最大化利润的前提下最小化成本$2x_1+4x_2$ <br>
cons:$x_1+x_2<=10$

In [13]:
from docplex.mp.model import Model
# 第一步：最大化利润
model1 = Model(name="第一次建模求取最大利润")
x1 = model1.continuous_var(name='x1',lb = 0)
x2 = model1.continuous_var(name="x2",lb = 0)
profit = 5*x1+3*x2
model1.maximize(profit)
model1.add_constraint(x1 + x2 <= 10)
solution1 = model1.solve()
best_profit = model1.objective_value # porfit 
print(f"Step 1: Optimal Profit = {best_profit}")

#第二步：在利润最优下最小化成本
model2 = Model(name="min cost")
x1 = model2.continuous_var(name="x1")
x2 = model2.continuous_var(name="x2")
profit = 5 * x1 + 3 * x2
cost = 2 * x1 + 4 * x2

model2.minimize(cost)
model2.add_constraint(x1 + x2 <= 10)
model2.add_constraint(profit>= best_profit) # 保证利润最优

solution2 = model2.solve()
if solution2:
    print("Step 2 Below:")
    print(f"x1 = {x1.solution_value}, x2 = {x2.solution_value}")
    print(f"Profit = {profit.solution_value}, Cost = {cost.solution_value}")


Step 1: Optimal Profit = 50.0
Step 2 Below:
x1 = 10.0, x2 = 0
Profit = 50.0, Cost = 20.0


## 软约束教学
数学建模见ob